In [3]:
import cv2
import numpy as np

#average pixel intensity in a region
def calculate_average_temperature(frame, x, y, x2, y2):
    region = frame[y:y2, x:x2]
    average_intensity = np.mean(region)
    return average_intensity

#mapping average intensity values to a realistic temperature range
def map_to_realistic_temperature(average_intensity):
    # Assuming average intensity ranges from 0 to 255
    min_intensity = 0
    max_intensity = 255

    # Assuming realistic temperature ranges from 36 to 38 degrees Celsius
    min_realistic_temp = 36
    max_realistic_temp = 39

    # Map the average intensity to the realistic temperature range
    realistic_temperature = min_realistic_temp + ((average_intensity - min_intensity) / (max_intensity - min_intensity)) * (max_realistic_temp - min_realistic_temp)

    return realistic_temperature


vid = cv2.VideoCapture(0)
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
last_detected_face = None
average_temperature_overall = []

while True:
    
    ret, frame = vid.read()

    if not ret:
        print("Error reading frame from the webcam.")
        break

    #grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces 
    bboxes = classifier.detectMultiScale(gray_frame)


    #bounding boxes on both frames
    for box in bboxes:
        x, y, width, height = box
        x2, y2 = x + width, y + height

        #bounding box on the normal frame
        thickness = 2
        cv2.rectangle(frame, (x, y), (x2, y2), (0, 0, 255), thickness)

        #bounding box on the "thermal" frame (applying a colormap)
        thermal_frame = cv2.applyColorMap(gray_frame, cv2.COLORMAP_JET)
        cv2.rectangle(thermal_frame, (x, y), (x2, y2), (255, 255, 255), thickness)
        last_detected_face = (x, y, x2, y2)

        #average temperature within the bounding box
        average_intensity = calculate_average_temperature(gray_frame, x, y, x2, y2)

        # Map average intensity to realistic temperature range
        realistic_temperature = map_to_realistic_temperature(average_intensity)
        average_temperature_overall.append(realistic_temperature)
        rf =(realistic_temperature * 9/5) + 32
        cv2.putText(thermal_frame, f'Temp: {rf:.2f} C', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # Resize "thermal" frame to match the normal frame's size
    thermal_frame_resized = cv2.resize(thermal_frame, (frame.shape[1], frame.shape[0]))

    # Display both frames side by side
    output_frame = np.hstack((frame, thermal_frame_resized))
    cv2.imshow('Normal and Thermal Simulation', output_frame)

    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == 27: 
        break

final_average_temperature = np.mean(average_temperature_overall)
fahrenheit = (final_average_temperature * 9/5) + 32
  

if last_detected_face is not None:
    print(f"Final Average Temperature: {fahrenheit:.2f} (fahrenheit)")
    print("Detected Face Coordinates:", last_detected_face)
else:
    print("No face detected.")

if fahrenheit > 100 :
    print("Might have fever")
    
    
vid.release()

cv2.destroyAllWindows()

Final Average Temperature: 99.91 (fahrenheit)
Detected Face Coordinates: (256, 161, 371, 276)
